In [ ]:
import uproot
import hist
import matplotlib.pyplot as plt
from scipy.interpolate import interp1d, interp2d
import numpy as np
from scipy.optimize import curve_fit
from matplotlib.colors import LogNorm
from time_templates.utilities.plot import add_identity, plot_profile_1d, plot_hist
import pandas as pd
from time_templates.datareader.get_data import fetch_MC_data_from_tree
from time_templates.utilities.fitting import plot_fit_curve
from time_templates.utilities import plot as ttplt
from time_templates.utilities.misc import histedges_equalN

PRIMARY_COLORS = {'proton': "b", 'iron': "r"}
HIM_MARKERS = {'EPOS_LHC': 'o', 'QGSJET-II.04': '^', 'SIB23': 'x'}
HIM_LS = {'EPOS_LHC': '-', 'QGSJET-II.04': '--', 'SIB23': ':'}
from matplotlib.lines import Line2D

him_marker_handles = [Line2D([0], [0], marker='o', color='grey', ls=''),
              Line2D([0], [0], marker='^', color='grey', ls='')]
him_ls_handles = [Line2D([0], [0], ls='-', color='grey'),
              Line2D([0], [0], ls='--', color='grey')]
him_labels = ['EPOS-LHC', 'QGSJet-II.04']
primary_handles = [Line2D([0], [0], color="#1D5287", lw=10),
                  Line2D([0], [0], color="#BC313D", lw=10)]
primary_labels = ['proton', 'iron']
plt.style.use('thesis')

In [ ]:
dfs = []

for HIM in ['EPOS_LHC', 'QGSJET-II.04']:
    for primary in ['proton', 'iron']:
        for energy in ['19_19.5']:
            df = fetch_MC_data_from_tree(HIM=HIM, primary=primary, energy=energy, dense=False, no_traces=True, force=False)
            df['HIM'] = HIM
            dfs.append(df)

df = pd.concat(dfs)
# df['shower_id'] = df['adstFilename'].str[-11:-5].astype('int')

# df = df.query('MCCosTheta > 0.6 & MClgE >= 19')
# df_mpd = pd.read_pickle(f'~/auger/data/time_templates/df_MPD.pl')
# df = df.reset_index().merge(df_mpd, on='shower_id').set_index(['EventId', 'StationId'])
df = df.groupby('EventId').apply(lambda x: x.iloc[0])

In [ ]:
f, ax = plt.subplots(1)
bins = np.linspace(400, 800)
for HIM, df_him in df.groupby('HIM'):
    for primary, df_prim in df_him.groupby('primary'):
        plot_hist(df_prim['Xmumax_1700'], bins=bins, histtype='step', color=PRIMARY_COLORS[primary], ls=HIM_LS[HIM], density=True, ax=ax)
ax.legend(him_ls_handles+primary_handles, him_labels+primary_labels)
ax.set_xlabel('Xmumax (r>1700m) [g/cm2]')

In [ ]:
f, ax = plt.subplots(1, 1, figsize=(5.5, 2.8))

def att_factor(sct, a, b, c=-200, d=0, cut=1.556):
    sct_ = sct - cut
    return a + b*sct_ + c*sct_**2 + d*sct_**3

bins = 1/np.sqrt(np.linspace(0.5**2, 1, 25))[::-1]
boots = 0
slopes = np.zeros(4)
slopes_err = np.zeros(4)
i = 0
for HIM, df_him in df.groupby('HIM'):
    for primary, df_prim in df_him.groupby('primary'):
        ax, (x, y, yerr) = plot_profile_1d(df_prim['MCSecTheta'], df_prim['Xmumax_1700'], bins=bins, ax=ax, bootstraps=boots,
                                           color=PRIMARY_COLORS[primary], marker=HIM_MARKERS[HIM])#,  weights=1/df_['Xmumax_1700_err']**2)
        mask = np.isfinite(x*y*yerr) & (x>1.3) & (x < 1.68)
        ax, (fitp_scale, fitp_err, _, _) = plot_fit_curve(x[mask], y[mask], func=att_factor, yerr=np.sqrt(yerr[mask]**2 + 0**2),  p0=[500, 100],
                                             ebar_kws=dict(color=PRIMARY_COLORS[primary], ls='', marker=HIM_MARKERS[HIM]), line_kws=dict(ls=HIM_LS[HIM]),
                                                   ax=ax, smoother_x=True)
#         print(fitp_scale)
        slopes[i] = fitp_scale[1]
        slopes_err[i] = fitp_err[1]
        i+=1
        
ax.axvspan(1, 1.3, color='k', hatch='//', alpha=0.3)
ax.axvspan(1.66, 2, color='k', hatch='\\\\', alpha=0.3)

ax.set_xlim([1, 2])
ax.set_ylim([400, 650])
slope = np.average(slopes, weights=1/slopes_err**2)
# slope = np.mean(slopes[:2])
print(slope)

# ax.legend()
ax.set_xlabel('$\sec \\theta$')
ax.set_ylabel('$X^\\mu_{\\rm max}(r > 1700\, \\rm m)\, [\\rm g/cm^2]$');
ax_top = ax.twiny()
ax_top.set_xlim(ax.get_xlim())
# new_tick_locations = np.arange(1, 2.1, 0.2)
new_tick_locations = np.array([1, 1/np.cos(np.deg2rad(30)), 1.3, 1/np.cos(np.deg2rad(45)), 1/np.cos(np.deg2rad(50)), 1.66, 2])
ax_top.set_xticks(new_tick_locations)
ax_top.minorticks_off()
new_tick_labels = np.rad2deg(np.arccos(1/new_tick_locations))
ax_top.set_xticklabels([int(round(l, 0)) for l in new_tick_labels])
ax_top.set_xlabel('$\\theta$ [deg]')
plt.savefig('/home/mart/auger/projects/time_templates/plots/Xmumax_1700_vs_theta.pdf', bbox_inches='tight')

f, ax = plt.subplots(1, 1, figsize=(5, 3))
df['Xmumax_50'] = df['Xmumax_1700']-att_factor(df['MCSecTheta'], 0, slope)

for HIM, df_him in df.groupby('HIM'):
    for primary, df_prim in df_him.groupby('primary'):
        ax, (x, y, yerr) = plot_profile_1d(df_prim['MCSecTheta'], df_prim['Xmumax_50'], bins=bins, ax=ax, 
                                          color=PRIMARY_COLORS[primary], marker=HIM_MARKERS[HIM])
ax.grid()
ax.axvspan(1, 1.3, color='k', hatch='//', alpha=0.3)
# ax.set_ylim([-50, 50])
# ax.fill_between([1.3, 1.65], -10, 10, color='k', alpha=0.2)
# ax.axhspan(-10, 10, color='k', alpha=0.2)
# ax.axhline(0, ls='--', color='k')
# ax.axvspan(1/0.6, 2., color='k', alpha=0.2)
ax.set_xlim([1, 1.65])
ax.legend(him_marker_handles+primary_handles, him_labels+primary_labels, loc=2)


In [ ]:
f, ax = plt.subplots(1, figsize=(9, 6))
bins = np.linspace(400, 750, 20)
for HIM, df_him in df.groupby('HIM'):
    for primary, df_prim in df_him.groupby('primary'):
        plot_hist(df_prim['Xmumax_50'], bins=bins, histtype='step', color=PRIMARY_COLORS[primary], ls=HIM_LS[HIM], density=True, ax=ax)
ax.legend(him_ls_handles+primary_handles, him_labels+primary_labels)
ax.set_xlabel('Xmumax (theta=50)(r>1700m) [g/cm2]')

In [ ]:
f, ax = plt.subplots(1, figsize=(4, 3.5))
a = []
a_err = []
slopes = []
slopes_err = []
lines = []
labels = []
for HIM, df_him in df.groupby('HIM'):
    him = HIM.replace('_', ' ')
    for primary, df_prim in df_him.groupby('primary'):
        df_prim.query('MCCosTheta < 0.77 & MCCosTheta > 0.6')
        ax, (x, y, yerr) = ttplt.plot_profile_1d(df_prim['MCXmax'], df_prim['Xmumax_50'], bins=np.linspace(650, 1000, 20),
                                                 color=PRIMARY_COLORS[primary], marker=HIM_MARKERS[HIM], ax=ax, alpha=0)
        mask = np.isfinite(x*y*yerr) & (yerr > 0)
        ax, (pfit, perr, _, _) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*(x-750), ax=ax,
                                                ebar_kws=dict(color=PRIMARY_COLORS[primary], marker=HIM_MARKERS[HIM], ls=''), line_kws=dict(ls=HIM_LS[HIM]))
        lines.append(Line2D([0], [0], color=PRIMARY_COLORS[primary], marker=HIM_MARKERS[HIM], ls=HIM_LS[HIM]))
        labels.append(f'{him} {primary} \n slope $= {pfit[1]:.2f} \\pm {perr[1]:.2f}$')
        slopes.append(pfit[1])
        slopes_err.append(perr[1])
        a.append(pfit[0])
        a_err.append(pfit[1])
        
slope = np.average(slopes, weights=1/np.array(slopes_err)**2)
a = np.average(a, weights=1/np.array(a_err)**2)
print(a, slope)
x = np.linspace(650, 950)
# ax.plot(x, a+slope*(x-750), 'k-')
# ax.legend(ncol=2)

ax.legend(lines, labels, loc=2)
ax.set_xlim([620, 1010])
ax.set_ylim([430, 850])
ax.set_xlabel('$X_{\\rm max} \, [\\rm g/cm^2]$')
ax.set_ylabel('$X^\\mu_{\\rm max}(\\theta = 50^\\circ,\, r > 1700\, \\rm m)\, [\\rm g/cm^2]$')
plt.savefig('/home/mart/auger/projects/time_templates/plots/Xmax_vs_Xmumax_theta40-53_r1700m.pdf', bbox_inches='tight')

In [ ]:
def Xmax_from_Xmumax_50(Xmumax_50):
    return 750 + (Xmumax_50-546)/0.693
f, ax = plt.subplots(1, figsize=(9, 7))
a = []
a_err = []
slopes = []
slopes_err = []
for HIM, df_him in df.groupby('HIM'):
    for primary, df_prim in df_him.groupby('primary'):
        plot_hist(df_prim['MCXmax'] - Xmax_from_Xmumax_50(df['Xmumax_50']), bins=np.linspace(-200, 200, 40), histtype='step',
                 color=PRIMARY_COLORS[primary], ls=HIM_LS[HIM], ax=ax, density=True)
ax.set_xlabel('Xmax - Xmax(Xmumax_50) [g/cm2]')
ax.set_xlim([-100, 100])

In [ ]:
ct2s = np.linspace(0.36, 1, 7)
a_fits = np.zeros((len(ct2s)-1))
b_fits = np.zeros((len(ct2s)-1))
a_errs = np.zeros((len(ct2s)-1))
b_errs = np.zeros((len(ct2s)-1))
for i, ct2 in enumerate(ct2s[:-1]):
    f, ax = plt.subplots(1)
    ct_l = np.sqrt(ct2)
    ct_r = np.sqrt(ct2s[i+1])
    a = 0
    b = 0
    aerr = 0
    berr = 0
#     for primary, df_ in df.groupby(['primary']):
    df_ = df.query(f'MCCosTheta >= {ct_l} & MCCosTheta < {ct_r} & primary == "proton" & HIM == "EPOS_LHC"')
    ax, (x, y, yerr) = ttplt.plot_profile_1d(df_['MCXmax'], df_['Xmumax_50'], bins=10, color=PRIMARY_COLORS[primary], ax=ax)
    mask = np.isfinite(x*y*yerr) & (yerr > 0)
    ax, (pfit, perr, _, _) = plot_fit_curve(x[mask], y[mask], yerr=yerr[mask], func=lambda x, a, b: a+b*(x-750), ax=ax, ebar_kws=dict(color=PRIMARY_COLORS[primary], ls=''))
    a = pfit[0]
    b = pfit[1]
    aerr = perr[0]#**2
    berr = perr[1]#**2
#     a /= 2
#     b /= 2
#     aerr = np.sqrt(aerr/2)
#     berr = np.sqrt(berr/2)
    a_fits[i] = a
    b_fits[i] = b
    a_errs[i] = aerr
    b_errs[i] = berr
    ax.legend()

In [ ]:
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))
ct2s_ = (ct2s[:-1]+ct2s[1:])/2

plot_fit_curve(1/np.sqrt(ct2s_), a_fits, yerr=a_errs, func=lambda x, a: a+0*(x-1.556), ax=ax1, smoother_x=True)
plot_fit_curve(1/np.sqrt(ct2s_), b_fits, yerr=b_errs, func=lambda x, a, b: a+b*(x-1.556), ax=ax2, smoother_x=True)

ax1.legend()
ax2.legend()

In [ ]:
def Xmumax_50_from_Xmax(Xmax, theta=0.87266):
    #EPOS LHC average between iron proton
    #determined in bins of cos^theta
    sct = 1 / np.cos(theta)
    a = 547.9
    b = 0.736 + 0.5 * (sct - 1.556)
    return a + b * (Xmax - 750)

df['Xmumax_50_pred'] = Xmumax_50_from_Xmax(df['MCXmax'], df['MCTheta'])

In [ ]:
df_ = df.query('HIM == "EPOS_LHC"')
ttplt.plot_hist(df_['Xmumax_50'] - df_['Xmumax_50_pred'], bins=np.linspace(-100, 100));

In [ ]:
f, ax = plt.subplots(1)
df_E = df.query('HIM == "EPOS_LHC"')
for primary, df_ in df_E.groupby(['primary']):
    ttplt.plot_profile_1d(df_['MClgE'], df_['Xmumax_50_pred']-df_['Xmumax_50'], bins=10, ax=ax, color=PRIMARY_COLORS[primary])
    

In [ ]:
f, ax = plt.subplots(1)
for primary, color in zip(['proton', 'iron'], ['b', 'r']):
    df_ = df.query(f"primary == '{primary}' & MCCosTheta < 0.8")
    
#     df_['Xmumax_1700_corr'] = df_['Xmumax_1700'] - att_factor(df_['MCSecTheta'], 0, 170)
    ax, (x, y, yerr) = plot_profile_1d(df_['MClgE'], df_['Xmumax_50'], bins=10, ax=ax, color=color)
    ax, (fitp_scale, _, _, _) = plot_fit_curve(x, y, func=lambda x, a, b: a+b*(x-np.log10(3e19)), yerr=np.sqrt(yerr**2 + 0**2),
                                         ebar_kws=dict(color=color, ls='', marker='o'), ax=ax, smoother_x=True) 
    
ax.legend()

In [ ]:

df['Xmumax_38'] = df['Xmumax_1000'] / att_factor(df['MCCosTheta']**2, *fitp_scale)
ax, _ = plot_profile_1d(df['MCSecTheta'], df['Xmumax_38'], bins=15);
# ax.scatter(np.sin(df['MCTheta'])**2, df['Xmumax_38'], marker='.', alpha=0.1)

# plot_profile_1d(df['MClgE'], df['Xmumax_38'], bins=10);

In [ ]:
tau = 2.2e-6 #sec
Ekin = 1000 #MeV
m = 105 #MeV
E = Ekin + m
gamma = E/m
beta = np.sqrt(1 - 1/gamma**2)
c = 2.998e8 #m/s
L = gamma * c * beta * tau
Xmumax = 650 #g/cm2
theta = 40 #deg
from time_templates.utilities.atmosphere import Atmosphere
atm = Atmosphere()
hground = 1400
h = atm.height_at_slant_depth(Xmumax, np.deg2rad(theta)) - hground
print(f"A muon with Ekin = {Ekin:.0f} MeV travels on average {L:.2f} m before decaying")
print(f"For Xmumax = {Xmumax:.0f} g/cm^2 and theta = {theta:.0f} deg the distance to the ground is {h:.1f} m")

In [ ]:
from fit_read_hist import Xmumax_from_hist_from_rootfile

In [ ]:
import glob

files = glob.glob('/home/mart/auger/data/UUB_sims_rec/MPD_new/MPD_EPOS_LHC*.root')
files

In [ ]:
Xmumax_from_hist_from_rootfile(np.random.choice(files), plot=True)

In [ ]:
import uproot
fl = uproot.open('/home/mart/auger/data/time_templates/MPD/MPD_EPOS_LHC_proton_18.5_19_DAT100055.root')

In [ ]:
h3d = fl[fl.keys()[0]].to_hist()

In [ ]:
h3d.axes[1].index(np.log10(1700))

In [ ]:
10**np.linspace(np.log10(400), np.log10(4000), 21), 10**lgr_bins

In [ ]:
(np.log10(1700)-np.log10(500))

In [ ]:
10**np.linspace(np.log10(400), np.log10(4000), 28)

In [ ]:
10**lgr_bins[h3d.axes[1].index(np.log10(4000))]

In [ ]:
(25 * 7 + 25)/(25+1)/240

In [ ]:
10/240

In [ ]:
vals, X_bins, lgr_bins, lgE_bins = h3d.to_numpy(flow=False)

idx = 12
plt.plot((X_bins[:-1]+X_bins[1:])/2, vals[:, idx:, ].sum(axis=(1, 2)))
h3d[:, idx:20:sum, sum].plot()
plt.xlim([300, 900])

In [ ]:
h_rebinned = h3d[::4j, ::2j, ::1j]
r_bins = h_rebinned.axes[1]
f, axes = plt.subplots(3, 3, figsize=(20, 20))
axes = axes.flatten()
for i in range(r_bins.size):
    try:
        ax = axes[i]
    except:
        continue
    ax.set_title(r_bins[i])
    h_rebinned[:80, i, :12].plot(ax=ax)
    ax.set_xlabel('X [g/cm2]')
    ax.set_ylabel('lgE/eV')